# Laboratorio 7
### Oscar Fernando López Barrios
### Carné 20679

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/malimg_paper_dataset_imgs"

In [15]:
!pip install adversarial-robustness-toolbox

In [19]:
import numpy as np
from tensorflow.keras.models import load_model
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

model = load_model('modelo_lab_7.h5')

classifier = KerasClassifier(model=model, clip_values=(0, 1))

attack = FastGradientMethod(estimator=classifier, eps=0.1)

data = ImageDataGenerator(rescale=1./255)
test_generator = data.flow_from_directory(path,
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

x_test_adv = []
y_true = []
for i in range(len(test_generator)):
    x, y = test_generator[i]
    x_adv = attack.generate(x=x)
    x_test_adv.append(x_adv[0])
    y_true.append(y[0])

y_pred_adv = classifier.predict(np.array(x_test_adv))

y_pred_adv_indices = np.argmax(y_pred_adv, axis=1)
y_true_indices = np.argmax(y_true, axis=1)

accuracy = np.mean(y_pred_adv_indices == y_true_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_adv_indices == y_true_indices) / len(y_true_indices)
print("Precision:", precision)

Found 9339 images belonging to 25 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Accuracy: 0.08234286326159118
Precision: 0.08234286326159118


In [20]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import DeepFool

model_logits = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

classifier = KerasClassifier(model=model_logits, use_logits=True, clip_values=(0, 1))

attack = DeepFool(classifier=classifier)

x_test, y_test = next(test_generator)
x_test_adv = attack.generate(x=x_test)

y_pred_adv = classifier.predict(x_test_adv)

y_pred_adv_indices = np.argmax(y_pred_adv, axis=1)
y_true_indices = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_adv_indices == y_true_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_adv_indices == y_true_indices) / len(y_true_indices)
print("Precision:", precision)

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.0
Precision: 0.0
